In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification

from IPython.display import Markdown

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [2]:
data = fetch_openml(data_id=1590, as_frame=True)

X_raw = data.data
y_raw = data.target
X = pd.get_dummies(X_raw)
y = 1 * (y_raw == ">50K")

c:\Users\I2044\Anaconda3\envs\python38\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
protected_attribute_name = f"sex_Male"

privileged_groups = [{protected_attribute_name: 1}]
unprivileged_groups = [{protected_attribute_name: 0}]

# Metric used (should be one of allowed_metrics)
metric_name = "Statistical parity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(42)

In [10]:
def _create_standard_dataset(
        df: pd.DataFrame,
        protected_attribute_name: str,
        label_name="class",
    ) -> StandardDataset:
        return StandardDataset(
            df,
            label_name=label_name,
            favorable_classes=[1],
            protected_attribute_names=[protected_attribute_name],
            privileged_classes=[[1]],
        )
# Step 1: Create the StandardDataset object
dataset = _create_standard_dataset(pd.concat([X, y], axis=1), protected_attribute_name)

# Step 2: train-test-val :: 15:3:3
dataset_tv, dataset_test = dataset.split([0.7], shuffle=True, seed=42)
dataset_train, dataset_valid = dataset_tv.split([0.7], shuffle=True, seed=42)

In [8]:
SCALER = StandardScaler()
X_train = SCALER.fit_transform(dataset_train.features)
y_train = dataset_train.labels.ravel()

LR = LogisticRegression()
LR.fit(X_train, y_train)

thresh = 0.5 
pos_ind = np.where(LR.classes_ == dataset_train.favorable_label)[0][0]

dataset_valid_pred = dataset_valid.copy(deepcopy=True)
X_valid = SCALER.fit_transform(dataset_valid_pred.features)
dataset_valid_pred.scores = LR.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)
dataset_valid_pred.labels = dataset_valid_pred.scores > thresh

ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups, 
    privileged_groups=privileged_groups, 
    low_class_thresh=0.01, high_class_thresh=0.99,
    num_class_thresh=100, num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub, metric_lb=metric_lb
)
ROC = ROC.fit(dataset_valid, dataset_valid_pred)

# user sends test data to their ML model
# the ML model makes predictions and computes prob
# The user then needs to send prob scores (biased predictions) + their test data to ROC_mitigator()
# we get unbiased predictions in return
dataset_test_pred = dataset_test.copy(deepcopy=True)
X_test = SCALER.fit_transform(dataset_test_pred.features)
y_test = dataset_test_pred.labels
dataset_test_pred.scores = LR.predict_proba(X_test)[:,pos_ind].reshape(-1,1)
dataset_test_pred.labels = dataset_test_pred.scores > thresh



# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_test_pred)

In [9]:
metric_unmit = ClassificationMetric(
    dataset_test,
    dataset_test_pred,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
)
metric_mit = ClassificationMetric(
    dataset_test,
    dataset_transf_test_pred,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
)
print(metric_unmit.disparate_impact())
print(metric_mit.disparate_impact())

0.2665362044165289
0.8774242659322757


In [6]:
metric_unmit = ClassificationMetric(
    dataset_test,
    dataset_test_pred,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
)
metric_unmit.disparate_impact()

0.2665362044165289

In [7]:
metric_mit = ClassificationMetric(
    dataset_test,
    dataset_transf_test_pred,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
)
metric_mit.disparate_impact()

0.8641017497800425